## LLM Clip

https://github.com/simonw/llm-clip
requires LLM: https://llm.datasette.io/en/stable/

```
$ pip install llm
$ llm install llm-clip
```

Then, assuming you are doing this in an environment (I create mine with conda), find the site packages directory, and the **llm_clip.py** file. `archaeology-images-ai/.venv/lib/python3.11/site-packages/llm_clip.py` is where mine hides.

For convenience, it would be nice to easily change models used in testing. For instance, you may have multiple models developed with different training data and different training parameters. For this reason, we should make a modification to the  **llm_clip.py**, by first adding to the top of the file this line:

```python
import os
```

And then alter this function to check to see if a `LLM_CLIP_MODEL_PATH` environment variable has been specified, and if so, use that:
le.

```python
def embed_batch(self, items):
    # Embeds a mix of text strings and binary images
    model_to_use = os.getenv('LLM_CLIP_MODEL_PATH', 'clip-ViT-B-32')
    # Now set the model, either from the env varaible or the default clip.
    if self._model is None:
        self._model = SentenceTransformer(model_to_use)
```

This should give some more flexibility in switching between different models to use with llm-clip.

In [1]:
import os
import requests

REPO_PATH = os.path.dirname(os.path.abspath(os.getcwd()))

# We will need files from:
# https://huggingface.co/sentence-transformers/clip-ViT-B-32/tree/main
# But since there's no clear license for these, we can't put them into source control in this repo.
# So we'll need to download them at run time and store in a folder that's ignored by git.
CLIP_FILE_WEB_ROOT = 'https://huggingface.co/sentence-transformers/clip-ViT-B-32/resolve/main'
REQUIRED_CLIP_FILES = [
    (None, 'config_sentence_transformers.json',),
    (None, 'modules.json'),
    ('0_CLIPModel', 'merges.txt', ),
    ('0_CLIPModel', 'preprocessor_config.json', ),
    ('0_CLIPModel', 'special_tokens_map.json', ),
    ('0_CLIPModel', 'tokenizer_config.json', ),
    ('0_CLIPModel', 'vocab.json', ),
]

def get_required_clip_files(use_model_path):
    """Populates the use_model directory with the needed CLIP files via
    Web requests from Hugging Face
    """
    for sub_dir, filename in REQUIRED_CLIP_FILES:
        if sub_dir:
            filepath =  os.path.join(use_model_path, sub_dir, filename)
            file_url = f'{CLIP_FILE_WEB_ROOT}/{sub_dir}/{filename}'
        else:
            filepath =  os.path.join(use_model_path, filename)
            file_url = f'{CLIP_FILE_WEB_ROOT}/{filename}'
        if os.path.exists(filepath):
            # We already have this file, no need to download it again.
            continue
        print(f'Get needed CLIP file from {file_url}')
        r = requests.get(file_url)  
        with open(filepath, 'wb') as fd:
            fd.write(r.content)

def move_results_data_into_use_model(results_path, use_model_path):
    """Moves the results data to the correct place for the use_mode"""
    clip_path =  os.path.join(use_model_path, '0_CLIPModel')
    os.makedirs(clip_path, exist_ok=True)
    files = ['pytorch_model.bin', 'config.json',]
    for file in files:
        source_path = os.path.join(results_path, file)
        use_path = os.path.join(clip_path, file)
        if os.path.exists(use_path):
            # nothing to do, we already have the expected
            # file in the correct place
            continue
        os.rename(source_path, use_path)
        print(f'Moved {file} to {use_path}')


def prepare_usable_model_from_fine_tuned_results(results_path, use_model_dir='fine_tuned_model'):
    use_model_path =  os.path.join(results_path, use_model_dir)
    # Make sure we have a directory for our overall model
    os.makedirs(use_model_path, exist_ok=True)
    # Now make sure that overall model has a directory called 0_CLIPModel
    clip_path =  os.path.join(use_model_path, '0_CLIPModel')
    os.makedirs(clip_path, exist_ok=True)
    # Make sure we have all the needed files.
    get_required_clip_files(use_model_path)
    # Make sure our results data is in the correct place.
    move_results_data_into_use_model(results_path, use_model_path)
    return use_model_path
    

In [2]:
import os

repo_path = os.path.dirname(os.path.abspath(os.getcwd()))
results_path = os.path.join(REPO_PATH, 'results')
# Set up our model for use
use_model_path = prepare_usable_model_from_fine_tuned_results(results_path)
os.environ['LLM_CLIP_MODEL_PATH'] = use_model_path

print(f'Using the model here: {os.getenv("LLM_CLIP_MODEL_PATH")}')

test_files = os.path.join(repo_path, 'files', 'testing')

# Load the test images into embeddings...
!llm embed-multi photos --files {test_files} '*.jpg' --binary -m clip

Get needed CLIP file from https://huggingface.co/sentence-transformers/clip-ViT-B-32/resolve/main/config_sentence_transformers.json
Get needed CLIP file from https://huggingface.co/sentence-transformers/clip-ViT-B-32/resolve/main/modules.json
Get needed CLIP file from https://huggingface.co/sentence-transformers/clip-ViT-B-32/resolve/main/0_CLIPModel/merges.txt
Get needed CLIP file from https://huggingface.co/sentence-transformers/clip-ViT-B-32/resolve/main/0_CLIPModel/preprocessor_config.json
Get needed CLIP file from https://huggingface.co/sentence-transformers/clip-ViT-B-32/resolve/main/0_CLIPModel/special_tokens_map.json
Get needed CLIP file from https://huggingface.co/sentence-transformers/clip-ViT-B-32/resolve/main/0_CLIPModel/tokenizer_config.json
Get needed CLIP file from https://huggingface.co/sentence-transformers/clip-ViT-B-32/resolve/main/0_CLIPModel/vocab.json
Moved pytorch_model.bin to /home/ekansa/github/archaeology-images-ai/results/fine_tuned_model/0_CLIPModel/pytorch_

In [28]:
# functions to look up images and captions from the results of llm search
import subprocess
import json
import IPython.display as display

def get_similar_ideas(query):
    # Construct the command
    cmd = f'llm similar photos -c "{query}"'
    
    # Execute the command and retrieve its output
    output = subprocess.check_output(cmd, shell=True)
    
    # Since the output is a bytes object, decode it to convert it to a string
    output_str = output.decode()
    
    # Split the output by lines and parse each line as JSON
    results = [json.loads(line) for line in output_str.split('\n') if line.strip()]
    
    # Return the parsed output
    return results

def get_similar_images(query):
    # Construct the command
    cmd = f'llm similar photos -i "{query}" --binary'
 
    # Execute the command and retrieve its output
    output = subprocess.check_output(cmd, shell=True)
    
    # Since the output is a bytes object, decode it to convert it to a string
    output_str = output.decode()
    
    # Split the output by lines and parse each line as JSON
    results = [json.loads(line) for line in output_str.split('\n') if line.strip()]
    
    # Return the parsed output
    return results

def get_image_metadata(image_ids):
    "Reads the json file and retrieves the metadata for the given image ids"
    metadata = {}
    test_path = os.path.join(REPO_PATH, 'files', 'test.json')
    with open(test_path, 'r') as f:
        for line in f:
            data = json.loads(line)
            if not data:
                continue
            if data['image'].replace("\\", "") in image_ids:
                metadata[data['image'].replace("\\", "")] = data['caption']
                
    return metadata

def display_similar_ideas(query):
    "Fetches similar ideas using the llm package and displays them with captions"
    similar_images = get_similar_ideas(query)
    testing_file_path = os.path.join(REPO_PATH, 'files', 'testing')
    image_ids = [os.path.join(testing_file_path, img['id']) for img in similar_images]
    image_metadata = get_image_metadata(image_ids)
    html_str = ''
    for img in similar_images:
        image_id = os.path.join(testing_file_path, img['id'])
        score = img['score']
        caption = image_metadata.get(image_id, '')  # Now this should retrieve the correct caption
        src = f"../files/testing/{ img['id']}"  # located in our repo, but under git version control
        url_id = img['id'].split('.jpg')[0]  # This will remove the '.jpg' from the image id
        url = "https://opencontext.org/media/" + url_id  # Create the URL by concatenating the base URL and the image id (without 'testing/' and '.jpg')
        
        html_str += f'<div><img src="{src}" width=25% alt="Image not found"> <p><strong>ID:</strong> {image_id}</p><p><strong>Score:</strong> {score}</p><p><strong>Caption:</strong> {caption}</p><p><a href="{url}">Link to full record</a></p></div>'
    display.display(display.HTML(html_str))

def display_similar_images(query):
    "Fetches similar ideas using the llm package and displays them with captions"
    similar_images = get_similar_images(query)
    testing_file_path = os.path.join(REPO_PATH, 'files', 'testing')
    image_ids = [os.path.join(testing_file_path, img['id']) for img in similar_images]
    image_metadata = get_image_metadata(image_ids)
    html_str = ''
    if '/files/' in query:
        q_file = query.split('/files/')[-1]
        q_src = f"../files/{ q_file}"
        html_str += f'<div>Search for images similar to:<br/><img src="{q_src}" width=25% alt="Image not found" /><p>----------------------------------</p>'
        html_str += '<p><strong>Results Below</strong>:</p></div>'
    for img in similar_images:
        image_id = os.path.join(testing_file_path, img['id'])
        score = img['score']
        caption = image_metadata.get(image_id, '')  # Now this should retrieve the correct caption
        src = f"../files/testing/{ img['id']}"  # located in our repo, but under git version control
        url_id = img['id'].split('.jpg')[0]  # This will remove the '.jpg' from the image id
        url = "https://opencontext.org/media/" + url_id  # Create the URL by concatenating the base URL and the image id (without 'testing/' and '.jpg')
        
        html_str += f'<div><img src="{src}" width=25% alt="Image not found"> <p><strong>ID:</strong> {image_id}</p><p><strong>Score:</strong> {score}</p><p><strong>Caption:</strong> {caption}</p><p><a href="{url}">Link to full record</a></p></div>'
    display.display(display.HTML(html_str))

In [29]:
query = "Bucchero pottery from Italy around 600 BCE"
display_similar_ideas(query)

# Assessment

I'm not really impressed with the results of the "idea" queries. Maybe we over-fit the data when training?

In [30]:
query = "/home/ekansa/github/archaeology-images-ai/files/training/73c2c55a-bed7-43b9-994d-3c1ac91a338a.jpg"
display_similar_images(query)